 # Statistical analysis (Terminos lagoon 2024)

In [1]:
# Import packages
import pandas as pd
import PyCO2SYS as pyco2
import scipy.stats as stats
# import scikit_posthocs as sp
from scipy.stats import kruskal
from tabulate import tabulate

In [2]:
# Function to read in csv file
def read_csv(file):
    """
    Read in csv file and return pandas dataframe
    """
    df = pd.read_csv(file, sep=",", header=0,  decimal=".", encoding='utf-8')
    return df

In [3]:
# define file path 
terminos_ta_dic_data_path = "../data/terminoslagoon_TA_DIC_2024_RawData.csv"

# Read in data
terminos_ta_dic_data = read_csv(terminos_ta_dic_data_path)

# Copy data to new dataframe
terminos_ta_dic = terminos_ta_dic_data.copy()

In [4]:
# Calculate TA - DIC (TA-DIC)  
terminos_ta_dic["TA-DIC"] = terminos_ta_dic["TA_micromol_kg"] - terminos_ta_dic["DIC_micromol_kg"]

### Calculate apparent oxygen utilization (AOU) and oxygen saturation (OS)

In [5]:
# Import function to convert oxygen concentration from mg/L to micromol/kg and calculate AOU
from convert_oxygen_concentration_AOU import convert_oxygen_concentration

# Import function to calculate AOU
from convert_oxygen_concentration_AOU import calculate_aou

# Convert DO from mg/L to micromol/kg
terminos_ta_dic["DO_micromol_kg"] = convert_oxygen_concentration(terminos_ta_dic["DO_mg_L"], 
                                                                terminos_ta_dic["Sal_psu"], terminos_ta_dic["Temp_C"], pressure=0) 
# Calculate Apparent Oxygen Utilization (AOU)
terminos_ta_dic["oxygen_utilization_micromol_kg"] = calculate_aou(terminos_ta_dic["Sal_psu"], terminos_ta_dic["Temp_C"],  
                                                                  0.0, terminos_ta_dic["DO_micromol_kg"], terminos_ta_dic["latitude"], 
                                                                  terminos_ta_dic["latitude"] )


## PyCO2SYS configuration from estuarine waters (Humphreys et al. 2022)

In [6]:
# Copy data to new dataframe
terminos_inorganic_carbon = terminos_ta_dic.copy()

# Create dictionary with data configuration for PyCO2SYS. 
# For more information on the parameters see Humphreys et al (2022)
my_co2sys_params_dic_ta_config = {
    # DIC measured in the lab in μmol/kg-sw
    "par1": terminos_inorganic_carbon["DIC_micromol_kg"],
    # TA measured in the lab, Total scale
    "par2": terminos_inorganic_carbon["TA_micromol_kg"],
    "par1_type": 2,         # tell PyCO2SYS: "par2 is a DIC value"
    "par2_type": 1,         # tell PyCO2SYS: "par1 is a TA value"
    # Fields conditions
    "salinity": terminos_inorganic_carbon["Sal_psu"],  # in-situ salinity in PSU
    # in-situ temperature (output conditions) in °C
    "temperature_out": terminos_inorganic_carbon["Temp_C"],
    "pressure_out": 0.0,
    # Settings
    "opt_pH_scale": 1,     # Total pH (Wolf-Gladrow et al. 2007)
    "opt_k_carbonic": 15,     # 0 < T < 50 °C, 1 < S < 50, Seawater scale, real seawater (Millero F.J. 2010)
    "temperature": 25      # lab temperature (input conditions) in °C
}

### Run PyCO2SYS

In [7]:
# Run PyCO2SYS to calculate the carbonate system parameters
terminos_inorganic_carbon_results= pyco2.sys(**my_co2sys_params_dic_ta_config)

### Read the interes variables from PYCO2sys 

In [8]:
# Select variables of interest from the PyCO2SYS output and create a DataFrame
pyco2_interest_variables = pd.DataFrame({
       "pH": terminos_inorganic_carbon_results["pH"],
       "saturation_aragonite": terminos_inorganic_carbon_results["saturation_aragonite"],
       "pCO2_atm": terminos_inorganic_carbon_results["pCO2_out"],
       "k_aragonite": terminos_inorganic_carbon_results["k_aragonite"],
       "total_calcium": terminos_inorganic_carbon_results["total_calcium"],
       "carbonate": terminos_inorganic_carbon_results["carbonate"]
       
})

# Concatenate PyCO2SYS output with the original dataframe
CarbonateTL = pd.concat([terminos_ta_dic, pyco2_interest_variables], axis=1)

### Select Candelaria and Palizada results

In [9]:
CandelariaResult = CarbonateTL.loc[CarbonateTL["Estuary"] == "Candelaria"]
                     
PalizadaResult = CarbonateTL.loc[CarbonateTL["Estuary"] == "Palizada"]

## Create statistical functions

In [10]:
# function to evaluate normality of variables, using the Shapiro-Wilk test
import scipy.stats as stats
from tabulate import tabulate

def evaluate_normality(variables):
    results = []
    
    for var_name, var_values in variables.items():
        stat, p_value = stats.shapiro(var_values)
        is_normal = p_value > 0.05
        
        results.append([var_name, stat, p_value, is_normal])
    
    print("Normality test results:")
    print(tabulate(results, headers=["Variable", "Statistic", "P-value", "Is Normal"], tablefmt="pretty"))
    
    return 

In [11]:
# Function to evaluate Mann-Whitney U test for two groups
from scipy.stats import mannwhitneyu

def evaluate_mannwhitney(data, variables, condition, group1, group2):
    results = []
    
    for var_name in variables:
        x = data[data[condition] == group1][var_name]
        y = data[data[condition] == group2][var_name]
    
        stat, p_value = mannwhitneyu(x, y)
        
        results.append([var_name, stat, p_value])
    
    print(f"Mann-Whitney U test for: {condition}, between {group1} and {group2}")
    print(tabulate(results, headers=["Variable", "Statistic", "P-value"], tablefmt="pretty"))
    
    return 


In [12]:
# Function to evaluate Kruskal-Wallis  and post hoc Dunn's test for three groups
# p<0.05 test is significant

from scipy.stats import kruskal
import scikit_posthocs as sp

def evaluate_kruskal_dunns(data, variables, condition, group1, group2, group3):
    results = []
    
    for var_name in variables:
        x = data[data[condition] == group1][var_name]
        y = data[data[condition] == group2][var_name]
        z = data[data[condition] == group3][var_name]
    
        # Kruskal-Wallis H-test
        stat, p_value = kruskal(x, y, z)
        
        # Post hoc Dunn's test             
        p_value_dunn_test = sp.posthoc_dunn([x, y, z], p_adjust="holm")
        
        p_value_dunn_test.columns = [group1, group2, group3]
        p_value_dunn_test.index = [group1, group2, group3]

                
        results.append([var_name, stat, p_value, p_value_dunn_test])
   

    print(f"Krustal-: {condition}, between {group1} and {group2}")
    print(tabulate(results, headers=["Variable", "Statistic", "P-value", "Post hoc Dunn's test"], tablefmt="pretty"))
    
        
    return 


### Evalute measured variables: Salinity, TA and DIC 

In [13]:
measured_variables_dict = {"Sal_psu": CarbonateTL["Sal_psu"],
                           "TA_micromol_kg": CarbonateTL["TA_micromol_kg"],
                           "DIC_micromol_kg": CarbonateTL["DIC_micromol_kg"],
                           "oxygen_utilization_micromol_kg": CarbonateTL["oxygen_utilization_micromol_kg"]
                           }  

In [14]:
# Evaluate normality of measured variables
evaluate_normality(measured_variables_dict)

Normality test results:
+--------------------------------+--------------------+------------------------+-----------+
|            Variable            |     Statistic      |        P-value         | Is Normal |
+--------------------------------+--------------------+------------------------+-----------+
|            Sal_psu             | 0.8885177323311573 |  3.41467921397059e-07  |   False   |
|         TA_micromol_kg         | 0.8611283026854756 | 2.462104840391832e-08  |   False   |
|        DIC_micromol_kg         | 0.8662748404427989 |  3.93332733607375e-08  |   False   |
| oxygen_utilization_micromol_kg | 0.869564601115901  | 5.3374299510246055e-08 |   False   |
+--------------------------------+--------------------+------------------------+-----------+


In [15]:
# Evaluate Mann-Whitney U test for Transects
evaluate_mannwhitney(CarbonateTL, measured_variables_dict, condition="Estuary", group1="Candelaria", group2="Palizada")

Mann-Whitney U test for: Estuary, between Candelaria and Palizada
+--------------------------------+-----------+------------------------+
|            Variable            | Statistic |        P-value         |
+--------------------------------+-----------+------------------------+
|            Sal_psu             |  1867.0   | 0.00014918883114817743 |
|         TA_micromol_kg         |  1280.5   |   0.8987921334386064   |
|        DIC_micromol_kg         |  1073.5   |  0.13031309768289984   |
| oxygen_utilization_micromol_kg |  1097.0   |  0.17524631410182823   |
+--------------------------------+-----------+------------------------+


In [16]:
# Evaluate Mann-Whitney U test for Seasons (Dry and Rainy)
evaluate_mannwhitney(CarbonateTL, measured_variables_dict, condition="Season", group1="Dry", group2="Rainy")

Mann-Whitney U test for: Season, between Dry and Rainy
+--------------------------------+-----------+------------------------+
|            Variable            | Statistic |        P-value         |
+--------------------------------+-----------+------------------------+
|            Sal_psu             |  1549.0   | 0.0033575510174735233  |
|         TA_micromol_kg         |  2138.5   | 8.706807858211458e-13  |
|        DIC_micromol_kg         |  1804.5   | 1.931836678910544e-06  |
| oxygen_utilization_micromol_kg |   487.0   | 3.2141094965536483e-06 |
+--------------------------------+-----------+------------------------+


In [17]:
# Evaluate Mann-Whitney U test for Layer depth (Surface and Bottom)
evaluate_mannwhitney(CarbonateTL, measured_variables_dict, condition="Layer_depth", group1="Surface", group2="Bottom")

Mann-Whitney U test for: Layer_depth, between Surface and Bottom
+--------------------------------+-----------+-----------------------+
|            Variable            | Statistic |        P-value        |
+--------------------------------+-----------+-----------------------+
|            Sal_psu             |  1154.5   |  0.34446722768419014  |
|         TA_micromol_kg         |  1258.0   |  0.8014934927731209   |
|        DIC_micromol_kg         |  1305.0   |  0.9545550270995352   |
| oxygen_utilization_micromol_kg |   861.0   | 0.0035793587980040426 |
+--------------------------------+-----------+-----------------------+


In [18]:
# Evaluate Kruskal-Wallis test for each Area: River, Estuary, and Coastal
evaluate_kruskal_dunns(CarbonateTL, measured_variables_dict, condition="Area", group1="River", group2="Estuarine", group3="Coast")

Krustal-: Area, between River and Estuarine
+--------------------------------+--------------------+------------------------+--------------------------------------------------+
|            Variable            |     Statistic      |        P-value         |               Post hoc Dunn's test               |
+--------------------------------+--------------------+------------------------+--------------------------------------------------+
|            Sal_psu             | 51.839704558234025 | 5.535427709577421e-12  |          River  Estuarine         Coast          |
|                                |                    |                        | River      1.000000e+00   0.000012  3.787329e-12 |
|                                |                    |                        | Estuarine  1.234044e-05   1.000000  7.356623e-03 |
|                                |                    |                        | Coast      3.787329e-12   0.007357  1.000000e+00 |
|         TA_micromol_kg        

In [19]:
# calcuated carbonate variables dictionary
carbonate_variables_dict= {"pCO2_atm": CarbonateTL["pCO2_atm"],
                           "pH": CarbonateTL["pH"],
                           "saturation_aragonite": CarbonateTL["saturation_aragonite"],
                           "TA-DIC" : CarbonateTL["TA-DIC"]
                           }

In [20]:
# Check normality of carbonate variables
evaluate_normality(carbonate_variables_dict)

Normality test results:
+----------------------+--------------------+------------------------+-----------+
|       Variable       |     Statistic      |        P-value         | Is Normal |
+----------------------+--------------------+------------------------+-----------+
|       pCO2_atm       | 0.7489856612270924 | 6.500111600563636e-12  |   False   |
|          pH          | 0.9826323648680874 |  0.20125161851756107   |   True    |
| saturation_aragonite | 0.9353591458909288 | 8.639813998846472e-05  |   False   |
|        TA-DIC        | 0.9454935436897395 | 0.00036629144140120306 |   False   |
+----------------------+--------------------+------------------------+-----------+


In [21]:
# Evaluate Mann-Whitney U test for Transects for carbonate variables
evaluate_mannwhitney(CarbonateTL, carbonate_variables_dict, condition="Estuary", group1="Candelaria", group2="Palizada")

Mann-Whitney U test for: Estuary, between Candelaria and Palizada
+----------------------+-----------+------------------------+
|       Variable       | Statistic |        P-value         |
+----------------------+-----------+------------------------+
|       pCO2_atm       |   872.0   | 0.0042139874500416565  |
|          pH          |  1614.0   |  0.03585506978505112   |
| saturation_aragonite |  1834.0   | 0.00035528706968920604 |
|        TA-DIC        |  1864.0   | 0.00016182543671919828 |
+----------------------+-----------+------------------------+


In [22]:
# Evaluate Mann-Whitney U test for Seasons (Dry and Rainy) for carbonate variables
evaluate_mannwhitney(CarbonateTL, carbonate_variables_dict, condition="Season", group1="Dry", group2="Rainy")

Mann-Whitney U test for: Season, between Dry and Rainy
+----------------------+-----------+-----------------------+
|       Variable       | Statistic |        P-value        |
+----------------------+-----------+-----------------------+
|       pCO2_atm       |   899.0   |  0.08734431848667445  |
|          pH          |  1487.0   |  0.01280103888797824  |
| saturation_aragonite |  1627.0   | 0.0004817888890513226 |
|        TA-DIC        |  1605.5   | 0.0008468925085571171 |
+----------------------+-----------+-----------------------+


In [23]:
# Evaluate Mann-Whitney U test for Layer depth (Surface and Bottom) 
evaluate_mannwhitney(CarbonateTL, carbonate_variables_dict, condition="Layer_depth", group1="Surface", group2="Bottom")

Mann-Whitney U test for: Layer_depth, between Surface and Bottom
+----------------------+-----------+---------------------+
|       Variable       | Statistic |       P-value       |
+----------------------+-----------+---------------------+
|       pCO2_atm       |  1440.0   | 0.33601473098677226 |
|          pH          |  1164.0   | 0.37798361955736226 |
| saturation_aragonite |  1175.0   | 0.41916533950066615 |
|        TA-DIC        |  1133.5   | 0.27742012731982835 |
+----------------------+-----------+---------------------+


In [24]:
# Evaluate Kruskal-Wallis test for each Area: River, Estuary, and Coastal
evaluate_kruskal_dunns(CarbonateTL, carbonate_variables_dict, condition="Area", group1="River", group2="Estuarine", group3="Coast")

Krustal-: Area, between River and Estuarine
+----------------------+-------------------+------------------------+-----------------------------------------------------+
|       Variable       |     Statistic     |        P-value         |                Post hoc Dunn's test                 |
+----------------------+-------------------+------------------------+-----------------------------------------------------+
|       pCO2_atm       | 39.67951680672269 | 2.4193706239238396e-09 |          River     Estuarine         Coast          |
|                      |                   |                        | River      1.000000e+00  2.766623e-07  6.890886e-08 |
|                      |                   |                        | Estuarine  2.766623e-07  1.000000e+00  6.600601e-01 |
|                      |                   |                        | Coast      6.890886e-08  6.600601e-01  1.000000e+00 |
|          pH          | 8.474327935057488 |  0.014448510236932973  |             River 

__________________